### This is ipynb notebook for generation of synthetic data using `paragraph.csv` and we can apped the generated questions to `input_questions.csv`.

In [1]:
from google.colab import drive

In [2]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
INPUT_PARAGRAPH = "paragraphs.csv"
INPUT_QUESTIONS = "input_questions.csv"

In [ ]:
!pip install -U transformers==3.0.0
!python -m nltk.downloader punkt
!git clone https://github.com/patil-suraj/question_generation.git

In [ ]:
%cd question_generation

In [ ]:
from pipelines import pipeline

In [ ]:
nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [ ]:
import pandas as pd

In [ ]:
df_question = pd.read_csv("/content/paragraphs.")

In [ ]:
df_new = pd.DataFrame({
    'theme':[],
    'paragraph':[],
    'question':[],
    'answer':[]
})
for ind, row in df_question.iterrows():
  if int(ind)%1000 == 0:
    print(ind)
  try:
    anss = nlp(str(row['paragraph']))
  except:
    continue
  for ans in anss:
    new_row = {
      'theme':row['theme'],
      'paragraph':row['paragraph'],
      'question':ans['question'],
      'answer':ans['answer']
    }
    df_new = df_new.append(new_row, ignore_index=True)

In [ ]:
df_new.to_csv("/content/synthetic_data.csv")

In [ ]:
%cd ..

In [ ]:
inp_para = pd.read_csv("paragraphs.csv")

In [ ]:
inp_qa = pd.read_csv("INPUT_QUESTIONS")

In [ ]:
syn_data = pd.read_csv("synthetic_data.csv")

In [ ]:
unique_paras = inp_para['paragraph']

In [ ]:
df_new = pd.DataFrame({
    'question':[],
    'theme':[],
    'paragraph_id':[],
    'answer':[]
})

for ind, row in syn_data.iterrows():
  para = row['paragraph']
  vect = {
      "question" : row["question"],
      "theme": row["theme"],
      "paragraph_id": int((inp_para.loc[inp_para["paragraph"] == para]).id),
      "answer" : row["answer"]
  }
  df_new = df_new.append(vect, ignore_index=True)
  inp_qa = inp_qa.append(vect, ignore_index=True)

In [ ]:
df_new.to_csv("synthetic_data_reformatted.csv")

In [ ]:
inp_qa = inp_qa.sort_values(by="paragraph_id")

In [ ]:
# inp_qa.drop(['id'], axis='columns')
inp_qa.head()

,question,theme,paragraph_id,answer
0,Which company produces the iPod?,IPod,1,Apple
1,When was the original iPod released?,IPod,1,"October 23, 2001"
2,How many different types of iPod are currently...,IPod,1,three
3,What kind of device is the iPod?,IPod,1,portable media players
4,The iPod Touch uses what kind of interface?,IPod,1,touchscreen


In [ ]:
df = pd.DataFrame({'id':[str(i) for i in range(1, len(inp_qa)+1)]})

In [ ]:
inp_qa['id'] = df['id']

In [ ]:
inp_qa.head()

,question,theme,paragraph_id,answer,id
0,Which company produces the iPod?,IPod,1,Apple,1
1,When was the original iPod released?,IPod,1,"October 23, 2001",2
2,How many different types of iPod are currently...,IPod,1,three,3
3,What kind of device is the iPod?,IPod,1,portable media players,4
4,The iPod Touch uses what kind of interface?,IPod,1,touchscreen,5


In [ ]:
inp_qa.to_csv("question_answer_all.csv", index=False)